**SVM Model for training**
-
The purpose of this notebook is to import direct RNA sequencing data and use it for training a machine learning model. This model is then optimized to get the final model. The data used in this notebook are `dataset0.json`, which contains the feature data, and `data.info.labelled`, which contains the actual labels. 

In [1]:
# Load the JSON file, reading each line and storing it as an element in a list
import gzip
import json

json_path = "dataset0.json.gz"

# Load in the feature data
data = []
with gzip.open(json_path, 'rt', encoding='utf-8') as f: 
    for line in f:
        if line.strip():  # skip empty lines
            record = json.loads(line)
            data.append(record)

In [2]:
# Load in the m6A labels
import pandas as pd

data_labels = pd.read_csv("data.info.labelled")

### Aggregation of feature reads data

In [3]:
# Let us compute the aggregate numbers for each line, using mean
import numpy as np

def aggregation(record):
    aggregates = []

    for transcript_id, value in record.items():
        for pos, value1 in value.items():
            for mers, value2 in value1.items():

                arr = np.array(value2, dtype=float)
                means = arr.mean(axis=0).tolist()
                aggregates.append({
                    "transcript_id": transcript_id,
                    "position": int(pos),
                    "kmer": mers,
                    "features": means
                })
    return aggregates

# For each record, use the function to "compress", for all reads, into 9 numbers, each representing one feature.
parsed_data = []
for line in data:
    parsed_data.extend(aggregation(line))

# Should give the same number of lines
print(len(parsed_data))

121838


In [4]:
# Data manipulation of the feature dataset to make it more readable
df_features = pd.DataFrame(parsed_data)
features_df = pd.DataFrame(df_features["features"].tolist(),
                           columns = [f"feature_{i+1}" for i in range(9)])
new_df = pd.concat([df_features.drop(columns=["features"]), features_df], axis = 1)

In [5]:
df_labels = pd.DataFrame(data_labels)

# Let us join the features and labels dataset together
ndf = pd.merge(new_df,
               df_labels,
               how = 'left',
               left_on = ['transcript_id', 'position'],
               right_on = ['transcript_id', 'transcript_position'])
print(len(ndf))
# Display the first few rows to understand what the data looks like
ndf.head()

121838


,transcript_id,position,kmer,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,gene_id,transcript_position,label
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,ENSG00000004059,244,0
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,ENSG00000004059,261,0
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,ENSG00000004059,316,0
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006102,2.236520,89.154000,ENSG00000004059,332,0
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,ENSG00000004059,368,0


### Splitting the data
We have a joined dataset, features and labels together. Now split the dataset into training and test sets, without having the same `gene_id` appear in both training and test set.

In [6]:
# Capture all the unique gene ids
unique_genes = ndf['gene_id'].unique()

In [7]:
# Split these genes into train and test genes in a 4:1 ratio, and ensure they do not reappear in either train or test data set
from sklearn.model_selection import train_test_split
train_gene, test_gene = train_test_split(unique_genes, test_size=0.2, random_state=42)

# Create train and test datasets
train_df = ndf[ndf['gene_id'].isin(train_gene)]
test_df = ndf[ndf['gene_id'].isin(test_gene)]

# Verify that there is indeed no repeated gene_id between train and test data sets
repeated_genes = set(train_df['gene_id']).intersection(set(test_df['gene_id']))
len(repeated_genes)

0

### Class imbalance
It is observed that the number of "0" records heavily dominates the number of "1" records in the training data. Need to fix this issue to prevent the trained model from becoming biased toward the majority class.

In [8]:
df_0 = train_df[train_df["label"] == 0]
# Randomly shuffle the rows
df0 = df_0.sample(frac=1, random_state=42).reset_index(drop=True)
df1 = train_df[train_df["label"] == 1]
print(f"Number of records labelled 0: {len(df0)}")
print(f"Number of records labelled 1: {len(df1)}")

Number of records labelled 0: 92368
Number of records labelled 1: 4471


## Performing hyperparameter tuning
This section performs hyperparameter tuning to identify the best combination of parameters that will make up the final model. Conducted on a subset of training data that has the number of "0" records to "1" records near a ratio of 1:1, for fast tuning.

In [15]:
# Hyperparameter tuning with a more balanced training set, with number of "0" records to "1" records near the ratio of 1:1
# Objective is to just find the best combination of parameters
df0_hyper = df0.iloc[:5000]
df_hyper = pd.concat([df0_hyper, df1], ignore_index=True)
# Randomly shuffle the rows for GridSearchCV
df_hyp = df_hyper.sample(frac=1, random_state=42).reset_index(drop=True)
df_hyp.head()

,transcript_id,position,kmer,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,gene_id,transcript_position,label
0,ENST00000422090,1225,GTGACCA,0.005712,4.888605,99.434884,0.008741,8.908372,114.883721,0.006148,2.933209,78.730233,ENSG00000213024,1225,0
1,ENST00000339775,1327,AGAACAC,0.008468,7.915185,130.777778,0.009431,3.790741,97.703704,0.007042,2.029519,91.914815,ENSG00000154640,1327,0
2,ENST00000301019,2278,TGGACAG,0.009758,3.337253,114.065934,0.010860,5.477363,112.450549,0.011198,3.327033,79.974725,ENSG00000167513,2278,1
3,ENST00000310931,4434,CAGACTC,0.006035,6.871818,106.885714,0.008658,4.614286,128.259740,0.007237,2.762338,89.302597,ENSG00000115677,4434,1
4,ENST00000307102,2558,TTAACCA,0.007056,1.938277,91.055319,0.006703,2.681553,92.653191,0.006615,1.823277,82.978723,ENSG00000169032,2558,0


In [16]:
# Hyperparameter Tuning of Support Vector Machine (SVM) with GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm

param_grid = {
    'C': [1, 10, 100],
    'gamma': [0.0001, 0.001, 0.01],
    'kernel': ['rbf', 'linear']
}

svc = svm.SVC(random_state=42)
model = GridSearchCV(svc, param_grid, refit=True, cv=5, verbose=2)

# Identify the features and labels from the training data
model.fit(df_hyp.iloc[:, 3:12], df_hyp["label"])
# Best parameters found by GridSearchCV, using a default scoring metric of accuracy
print("Best Parameters:", model.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ......................C=1, gamma=0.0001, kernel=rbf; total time=   2.1s
[CV] END ......................C=1, gamma=0.0001, kernel=rbf; total time=   2.0s
[CV] END ......................C=1, gamma=0.0001, kernel=rbf; total time=   2.1s
[CV] END ......................C=1, gamma=0.0001, kernel=rbf; total time=   2.0s
[CV] END ......................C=1, gamma=0.0001, kernel=rbf; total time=   2.1s
[CV] END ...................C=1, gamma=0.0001, kernel=linear; total time=   9.0s
[CV] END ...................C=1, gamma=0.0001, kernel=linear; total time=   8.5s
[CV] END ...................C=1, gamma=0.0001, kernel=linear; total time=  15.0s
[CV] END ...................C=1, gamma=0.0001, kernel=linear; total time=   8.8s
[CV] END ...................C=1, gamma=0.0001, kernel=linear; total time=   9.1s
[CV] END .......................C=1, gamma=0.001, kernel=rbf; total time=   1.4s
[CV] END .......................C=1, gamma=0.001

### Training the final model
With the best combination of parameters from hyperparameter tuning, train the final model

In [9]:
# Hyperparameter tuning with a slightly less balanced training set, with number of "0" records to "1" records near the ratio of 4:1
df0_sub = df0.iloc[:16000]
df_final = pd.concat([df0_sub, df1], ignore_index=True)

In [18]:
from sklearn.svm import SVC

# Train SVM on the training data, with the best combination of parameters from tuning (C:100, gamma:0.01, kernel:'rbf')
svm_model = SVC(kernel='rbf', C = 100, gamma = 0.01, class_weight='balanced',
                probability = True, random_state=42)
# Identify the features and labels, and fit the model to them
svm_model.fit(df_final.iloc[:, 3:12], df_final["label"])

# After training the model, we can test on our validation set
y_prob = svm_model.predict_proba(test_df.iloc[:, 3:12])
y_pred = svm_model.predict(test_df.iloc[:, 3:12])

### Evaluation on test set

In [19]:
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, f1_score, precision_score, recall_score, confusion_matrix

print("SVM Accuracy:", accuracy_score(test_df["label"], y_pred))
print("ROC_AUC Score:", roc_auc_score(test_df["label"], y_prob[:, 1]))
print("PR_AUC Score:", average_precision_score(test_df["label"], y_prob[:, 1]))
print("F1 Score:", f1_score(test_df["label"], y_pred))
print("Precision:", precision_score(test_df["label"], y_pred))
print("Recall:", recall_score(test_df["label"], y_pred))

SVM Accuracy: 0.8262730509220368
ROC_AUC Score: 0.8627600869703101
PR_AUC Score: 0.32257460536880667
F1 Score: 0.2584941096124296
Precision: 0.1559859880486297
Recall: 0.7539840637450199


In [20]:
# Confusion matrix on test set
print(confusion_matrix(test_df["label"], y_pred))

[[19899  4096]
 [  247   757]]


In [21]:
# Save the final model as a .pkl file
import pickle

with open("svm_train.pkl", "wb") as f:
    pickle.dump(svm_model, f)

### Training a second SVM
Training a second model to produce a second set of predictions, with the best combination of parameters too. But this time, instead of the first 16000 records of the shuffled `df0`, extract the last 16000 records of it.

In [13]:
df0_sub = df0.iloc[-16000:]
df_final = pd.concat([df0_sub, df1], ignore_index=True)

In [14]:
from sklearn.svm import SVC

# Train SVM on the training data, with the best combination of parameters from tuning (C:100, gamma:0.01, kernel:'rbf')
svm_model = SVC(kernel='rbf', C=100, gamma=0.01, class_weight='balanced',
                probability=True, random_state=1)
# Identify the features and labels, and fit the model to them
svm_model.fit(df_final.iloc[:, 3:12], df_final["label"])

# After training the model, we can test on our validation set
y_prob = svm_model.predict_proba(test_df.iloc[:, 3:12])
y_pred = svm_model.predict(test_df.iloc[:, 3:12])

In [15]:
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, f1_score, precision_score, \
    recall_score, confusion_matrix

print("SVM Accuracy:", accuracy_score(test_df["label"], y_pred))
print("ROC_AUC Score:", roc_auc_score(test_df["label"], y_prob[:, 1]))
print("PR_AUC Score:", average_precision_score(test_df["label"], y_prob[:, 1]))
print("F1 Score:", f1_score(test_df["label"], y_pred))
print("Precision:", precision_score(test_df["label"], y_pred))
print("Recall:", recall_score(test_df["label"], y_pred))

SVM Accuracy: 0.8284331373254931
ROC_AUC Score: 0.8598173673300131
PR_AUC Score: 0.3095739442981673
F1 Score: 0.2616629368221725
Precision: 0.15816857440166493
Recall: 0.7569721115537849


In [16]:
# Confusion matrix on test set
print(confusion_matrix(test_df["label"], y_pred))

[[19950  4045]
 [  244   760]]


Comparing the evaluation metrics of both SVM models, the second model appears to be more accurate in predicting true positives and true negatives, and is slightly preferred to the first model.

In [17]:
# Save this model as a .pkl file too
import pickle

with open("svm_train_2.pkl", "wb") as f:
    pickle.dump(svm_model, f)